In [9]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from datetime import date, datetime as dt, timedelta as td
import pandas as pd

In [10]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

In [11]:
SPREADSHEET_ID = "1anv8KVrybPKsdylq_VdnBykKZy4fdqHdT7Mves0naBA"
RANGE_NAME = "transactions_list_table"

In [12]:
# https://stackoverflow.com/questions/11485271/google-oauth-2-authorization-error-redirect-uri-mismatch
# Authentication
creds = None

# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.

if os.path.exists('data/token.pickle'):
    with open('data/token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('data/credentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# build service
service = build('sheets', 'v4', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=508154319195-fpk2traq6grkafl4h3uvtan41s6nattp.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=M3Y7Jnw7bDZwbHBXpqZylKfgqgNhdI&access_type=offline


In [13]:
# Get Data from Google Sheet and Return a List
def get_gsheet_data(SPREADSHEET_ID, RANGE_NAME):
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])
    return values

In [14]:
# get data
data_list = get_gsheet_data(SPREADSHEET_ID, RANGE_NAME)
# type(data) #list

In [15]:
# create data and frame and use first row as header
data = pd.DataFrame(data_list)
new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

In [16]:
data

,Type,Date,Set Time,Title,Amount,Currency,Exchange Rate,Category Group Name,Category,Account,Notes,Labels,Status
1,Transfer,2021-07-06 18:12:14,18:12,Vanguard Brokerage Account,-100,USD,1,(Transfer),(Transfer),Landmark Bank Checking,,,None
2,Transfer,2021-07-06 18:12:14,18:12,Vanguard Brokerage Account,100,USD,1,(Transfer),(Transfer),Vanguard Brokerage Account,,,None
3,Expense,2021-07-03 10:08:03,10:08,Digital Ocean,-1.04,USD,1,Entertainment,Others,Chase Sapphire,,,None
4,Expense,2021-07-02 12:14:09,12:14,Ameren,-26.05,USD,1,Rent and Utilities,Gas,Landmark Bank Checking,,,None
5,Expense,2021-07-01 7:00:47,7:00,GiveWell,-33,USD,1,Others,Charity,Chase Sapphire,,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218,New Account,2018-10-17 9:43:06,9:43,Landmark Bank Checking,7753.08,USD,1,(New Account),(New Account),Landmark Bank Checking,,,Reconciled
1219,New Account,2018-10-17 9:41:46,9:41,TSP,5753.93,USD,1,(New Account),(New Account),TSP,,,Reconciled
1220,New Account,2018-10-17 9:39:32,9:39,Ally Bank Savings,10000,USD,1,(New Account),(New Account),Ally Bank Savings,,,Reconciled
1221,New Account,2018-10-15 17:15:50,17:15,Barclaycard,-666.5,USD,1,(New Account),(New Account),Barclaycard,,,Reconciled


In [17]:
# export to csv
data.to_csv("data/budget.csv", index=False)